In [59]:
import pandas as pd
import re

# Cargar dataset
df = pd.read_csv("../output/dataset_farmacia_integrado.csv")

# Normalizar nombres
df["nombre_normalizado"] = df["nombre_normalizado"].fillna("").str.lower()

# -------------------------------------------------------------
# 1. DICCIONARIO DE PALABRAS CLAVE → CATEGORÍA
#    (Después yo te puedo generar uno completo leyendo tu dataset)
# -------------------------------------------------------------

diccionario = {
    # --- Medicamentos ---
    "amoxicilina": "Antibióticos",
    "azitromicina": "Antibióticos",
    "claritromicina": "Antibióticos",
    "diclofenaco": "Analgésicos y antiinflamatorios",
    "ibuprofeno": "Analgésicos y antiinflamatorios",
    "paracetamol": "Analgésicos y antiinflamatorios",
    "naproxeno": "Analgésicos y antiinflamatorios",
    "ketorolaco": "Analgésicos y antiinflamatorios",
    "flavicold": "Antigripales y antitusivos",
    "mentisan": "Antigripales y antitusivos",
    "loratadina": "Antialérgicos",
    "cetirizina": "Antialérgicos",
    "omeprazol": "Gastrointestinales",
    "ranitidina": "Gastrointestinales",
    "losartan": "Cardiovasculares",
    "enalapril": "Cardiovasculares",
    "clonazepam": "Sistema nervioso",
    "diazepam": "Sistema nervioso",
    "albendazol": "Antiparasitarios",
    "metronidazol": "Antiparasitarios",
    "ketoconazol": "Antifúngicos y dermatológicos medicamentosos",
    "miconazol": "Antifúngicos y dermatológicos medicamentosos",
    "vitamina": "Vitaminas y suplementos",
    "suplemento": "Vitaminas y suplementos",
    "shampoo": "Cuidado personal y cosméticos",
    "crema": "Cuidado personal y cosméticos",
    "jabon": "Cuidado personal y cosméticos",
    "jeringa": "Insumos médicos",
    "gasa": "Insumos médicos",
    "guante": "Insumos médicos",
    "venda": "Insumos médicos",
    "preservativo": "Higiene sexual y reproductiva",
    "condon": "Higiene sexual y reproductiva",
    "lubricante": "Higiene sexual y reproductiva",
    "ensure": "Nutrición y alimentos especiales",
    "pediasure": "Nutrición y alimentos especiales",
}

# -------------------------------------------------------------
# 2. FUNCIÓN PARA CLASIFICAR
# -------------------------------------------------------------

def clasificar_producto(nombre):
    for palabra, categoria in diccionario.items():
        if palabra in nombre:
            return categoria
    return "Varios / misceláneos"

# -------------------------------------------------------------
# 3. APLICAR CLASIFICACIÓN
# -------------------------------------------------------------

df["categoria"] = df["nombre_normalizado"].apply(clasificar_producto)

# -------------------------------------------------------------
# 4. GUARDAR RESULTADO
# -------------------------------------------------------------

df.to_csv("dataset_clasificado.csv", index=False)

print("Clasificación completada. Archivo guardado como dataset_clasificado.csv")


Clasificación completada. Archivo guardado como dataset_clasificado.csv
